In [3]:
import os
import numpy as np
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
from tkinter import Tk, filedialog

# コンスタントの定義
VU_REF_DB = -18.0
VU_MAX = 0.0
REF_LEVEL = 10**(VU_REF_DB / 20)

# ファイル選択ダイアログを表示してファイルを選択する
root = Tk()
root.withdraw()  # メインウィンドウを表示しない
file_path = filedialog.askopenfilename(filetypes=[("WAV files", "*.wav")])
root.destroy()  # Tkinterウィンドウを破棄する

# オーディオファイルの読み込み
if file_path:
    rate, data = wav.read(file_path)
    if data.ndim == 1:
        data = np.expand_dims(data, axis=1)  # モノラルファイルの場合、チャンネルを追加

    # サンプルデータを0～1の範囲に正規化
    normalized_data = data / np.max(np.abs(data), axis=0)

    # VUメーターフィルタ（約300msのリリースタイム）
    def vu_meter(signal, rate, release_time=0.3):
        alpha = np.exp(-1.0 / (release_time * rate))
        vu_level = np.zeros_like(signal)
        for i in range(1, len(signal)):
            vu_level[i] = max(alpha * vu_level[i-1], np.abs(signal[i]))
        return vu_level

    # VUメーターのレベルを計算
    vu_levels = np.zeros_like(normalized_data)
    for ch in range(normalized_data.shape[1]):
        vu_levels[:, ch] = vu_meter(normalized_data[:, ch], rate)

    # 各チャンネルの最大レベルをチェックし、統一したゲインを計算
    max_vu = np.max(vu_levels)
    gain_adjustment = VU_MAX / max_vu if max_vu > REF_LEVEL else 1.0

    # 統一したゲインを適用
    adjusted_data = normalized_data * gain_adjustment

    # 新しいファイル名を生成
    output_path = os.path.splitext(file_path)[0] + '_VUoutput.wav'

    # 調整後のオーディオデータを保存
    adjusted_data_int = np.int16(adjusted_data * np.iinfo(np.int16).max)
    wav.write(output_path, rate, adjusted_data_int)
    print(f"Adjusted audio saved as '{output_path}'")

    # プロットで結果を表示
    plt.figure(figsize=(10, 4))
    plt.plot(normalized_data[:, 0], label='Original Left')
    if normalized_data.shape[1] > 1:
        plt.plot(normalized_data[:, 1], label='Original Right')
    plt.plot(adjusted_data[:, 0], label='Adjusted Left')
    if adjusted_data.shape[1] > 1:
        plt.plot(adjusted_data[:, 1], label='Adjusted Right')
    plt.legend()
    plt.title('Audio Waveform')
    plt.show()

    plt.figure(figsize=(10, 4))
    plt.plot(vu_levels[:, 0], label='VU Levels Left')
    if vu_levels.shape[1] > 1:
        plt.plot(vu_levels[:, 1], label='VU Levels Right')
    plt.axhline(y=REF_LEVEL, color='r', linestyle='--', label='0 VU (-18 dBFS)')
    plt.legend()
    plt.title('VU Meter Levels')
    plt.show()
    